In [1]:
import pandas as pd
import jsonlines

In [2]:
qa_set_path = './QA증강_intent.csv'
retriever_train_path = './qa_retriever_train.jsonl'
candidate_pool_path = './candidate_pool_add_qa.jsonl'
sft_dialog_path = './dialog_sft.json'

In [3]:
## QA 데이터셋 불러오기
qa_set = pd.read_csv('./QA증강_intent.csv', encoding = 'cp949')

In [4]:
qa_set

,질문,이상적인 답변,관련근거
0,휴가비 나오는 기준이 뭔가요,휴가여비 지급기준에 대해 안내해 드리겠습니다.\n\n휴가비 지급액 산출 방법은 출발...,"육군규정120 병영생활규정\n제106조(연가) 제3항,\n22년 육군 예산회계 실무지침"
1,휴가비는 어떤 기준으로 책정되나요?,휴가여비 지급기준에 대해 안내해 드리겠습니다.\n\n휴가비 지급액 산출 방법은 출발...,"육군규정120 병영생활규정\n제106조(연가) 제3항,\n22년 육군 예산회계 실무지침"
2,군인 휴가비 계산 방식이 궁금합니다. 어떻게 되나요?,휴가여비 지급기준에 대해 안내해 드리겠습니다.\n\n휴가비 지급액 산출 방법은 출발...,"육군규정120 병영생활규정\n제106조(연가) 제3항,\n22년 육군 예산회계 실무지침"
3,휴가비 지급 기준에 대해 설명해주세요.,휴가여비 지급기준에 대해 안내해 드리겠습니다.\n\n휴가비 지급액 산출 방법은 출발...,"육군규정120 병영생활규정\n제106조(연가) 제3항,\n22년 육군 예산회계 실무지침"
4,군 휴가비는 어떻게 산정되는지 알 수 있을까요?,휴가여비 지급기준에 대해 안내해 드리겠습니다.\n\n휴가비 지급액 산출 방법은 출발...,"육군규정120 병영생활규정\n제106조(연가) 제3항,\n22년 육군 예산회계 실무지침"
...,...,...,...
2360,전직지원교육에 대해 알려줘,"흔히 '직보반'이라 불리는 '전직 및 위업지원교육'에 대해 알려드릴게요.\n\n""전...","군 전직 및 취업 지원 업무에 \n관한 국방부 훈령,\n전역장병 지원에 관한 육군규..."
2361,대민지원 꼭 나가야해?,"대민지원이란 국민의 재난예방 및 복구, 농촌 일손돕기, 구난구조, 국가정책사업 및 ...",육군 행정예규 제22호 대민지원 업무처리예규
2362,전역할때 가져갈 수 있는게 뭐야?,전역 시 휴대해야할 품목 안내 입니다.\n\n전역자는 개인피복으로 지급받은 모든 품...,NaN
2363,보급품 잃어버리면 다시 받을 수 있나요?,"개인 피복, 개인 장구류 등 보급품 분실에 대해 안내해 드릴게요.\n\n보충피복비를...",24년 물자보급 및 운영규정


In [27]:
## retriever train set을 만들기 위한 과정
retriever_train = []
for index, row in qa_set.iterrows():
    retriever_train.append({"source": row["질문"], "target": row["이상적인 답변"]})

with jsonlines.open(retriever_train_path, mode = 'w') as writer:
    writer.write_all(retriever_train)


In [6]:
answer_list = qa_set[['이상적인 답변', '관련근거']].drop_duplicates()
candidate_pool = []
for index, row in answer_list.iterrows():
    candidate_pool.append({"subject": "", "text": row['이상적인 답변'], "reference": row['관련근거']})

with jsonlines.open(candidate_pool_path, mode = 'w') as writer:
    writer.write_all(candidate_pool)


In [5]:
## candidate_pool을 만들기 위한 과정
answer_list = qa_set[['이상적인 답변', '관련근거']].drop_duplicates()
candidate_pool = []
for row in answer_list:
    candidate_pool.append({"subject": "", "text": row, "reference": ""})

with jsonlines.open(candidate_pool_path, mode = 'w') as writer:
    writer.write_all(candidate_pool)


AttributeError: 'DataFrame' object has no attribute 'to_list'

In [10]:
qa_set["이상적인 답변 종합"] = qa_set["이상적인 답변"] + "\n ※관련 근거:" + qa_set['관련근거'].fillna('')

In [9]:
qa_set['관련근거'].fillna('')

0        육군규정120 병영생활규정\n제106조(연가) 제3항,\n22년 육군 예산회계 실무지침
1        육군규정120 병영생활규정\n제106조(연가) 제3항,\n22년 육군 예산회계 실무지침
2        육군규정120 병영생활규정\n제106조(연가) 제3항,\n22년 육군 예산회계 실무지침
3        육군규정120 병영생활규정\n제106조(연가) 제3항,\n22년 육군 예산회계 실무지침
4        육군규정120 병영생활규정\n제106조(연가) 제3항,\n22년 육군 예산회계 실무지침
                              ...                        
2360    군 전직 및 취업 지원 업무에 \n관한 국방부 훈령,\n전역장병 지원에 관한 육군규...
2361                             육군 행정예규 제22호 대민지원 업무처리예규
2362                                                     
2363                                      24년 물자보급 및 운영규정
2364                                                     
Name: 관련근거, Length: 2365, dtype: object

In [7]:
qa_set["이상적인 답변 종합"]

0       휴가여비 지급기준에 대해 안내해 드리겠습니다.\n\n휴가비 지급액 산출 방법은 출발...
1       휴가여비 지급기준에 대해 안내해 드리겠습니다.\n\n휴가비 지급액 산출 방법은 출발...
2       휴가여비 지급기준에 대해 안내해 드리겠습니다.\n\n휴가비 지급액 산출 방법은 출발...
3       휴가여비 지급기준에 대해 안내해 드리겠습니다.\n\n휴가비 지급액 산출 방법은 출발...
4       휴가여비 지급기준에 대해 안내해 드리겠습니다.\n\n휴가비 지급액 산출 방법은 출발...
                              ...                        
2360    흔히 '직보반'이라 불리는 '전직 및 위업지원교육'에 대해 알려드릴게요.\n\n"전...
2361    대민지원이란 국민의 재난예방 및 복구, 농촌 일손돕기, 구난구조, 국가정책사업 및 ...
2362                                                  NaN
2363    개인 피복, 개인 장구류 등 보급품 분실에 대해 안내해 드릴게요.\n\n보충피복비를...
2364                                                  NaN
Name: 이상적인 답변 종합, Length: 2365, dtype: object

In [11]:
qa_set.tail()

,질문,이상적인 답변,관련근거,이상적인 답변 종합
2360,전직지원교육에 대해 알려줘,"흔히 '직보반'이라 불리는 '전직 및 위업지원교육'에 대해 알려드릴게요.\n\n""전...","군 전직 및 취업 지원 업무에 \n관한 국방부 훈령,\n전역장병 지원에 관한 육군규...","흔히 '직보반'이라 불리는 '전직 및 위업지원교육'에 대해 알려드릴게요.\n\n""전..."
2361,대민지원 꼭 나가야해?,"대민지원이란 국민의 재난예방 및 복구, 농촌 일손돕기, 구난구조, 국가정책사업 및 ...",육군 행정예규 제22호 대민지원 업무처리예규,"대민지원이란 국민의 재난예방 및 복구, 농촌 일손돕기, 구난구조, 국가정책사업 및 ..."
2362,전역할때 가져갈 수 있는게 뭐야?,전역 시 휴대해야할 품목 안내 입니다.\n\n전역자는 개인피복으로 지급받은 모든 품...,NaN,전역 시 휴대해야할 품목 안내 입니다.\n\n전역자는 개인피복으로 지급받은 모든 품...
2363,보급품 잃어버리면 다시 받을 수 있나요?,"개인 피복, 개인 장구류 등 보급품 분실에 대해 안내해 드릴게요.\n\n보충피복비를...",24년 물자보급 및 운영규정,"개인 피복, 개인 장구류 등 보급품 분실에 대해 안내해 드릴게요.\n\n보충피복비를..."
2364,국외 위탁가는데 생활지원금 나와?,위탁교육생 생활지원금에 대해 안내해드릴게요.\n\n국외 위탁생 중 6개월 이상의 교...,NaN,위탁교육생 생활지원금에 대해 안내해드릴게요.\n\n국외 위탁생 중 6개월 이상의 교...


In [18]:
sft_dialog = []
qa_set["이상적인 답변 종합"] = qa_set["이상적인 답변"] + "\n ※관련 근거:" + qa_set["관련근거"].fillna('')
for index, row in qa_set.iterrows():
    sft_dialog.append([
        {"role": "system", "content" : "시스템 메세지를 넣으세요"}, ## 시스템 메세지 넣기
        {"role":"user", "content":row["질문"]}, 
        {"role":"assistant", "content":row["이상적인 답변 종합"]}
    ])

In [9]:
## dialog_templete 만들기
sft_dialog = []
for index, row in qa_set.iterrows():
    sft_dialog.append([
        {"role": "system", "content" : "시스템 메세지를 넣으세요"}, ## 시스템 메세지 넣기
        {"role":"user", "content":row["질의문"]}, 
        {"role":"assistant", "content":row["답변"]}
    ])


# with jsonlines.open(sft_dialog_path, mode = 'w') as writer:
#     writer.write_all(sft_dialog)

with open(sft_dialog_path, 'w', encoding='utf-8') as file:
    json.dump(sft_dialog, file, ensure_ascii=False)